In [1]:
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from collections import OrderedDict, namedtuple, defaultdict


def get_auc(loader, model):
    pred, target = [], []
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device).float(), y.to(device).float()
            y_hat = model(x)
            pred += list(y_hat.cpu().numpy())
            target += list(y.cpu().numpy())
    auc = roc_auc_score(target, pred)
    return auc


class DNN(nn.Module):
    def __init__(self, inputs_dim, hidden_units, dropout_rate, ):
        super(DNN, self).__init__()
        self.inputs_dim = inputs_dim
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate)

        self.hidden_units = [inputs_dim] + list(self.hidden_units)
        self.linear = nn.ModuleList([
            nn.Linear(self.hidden_units[i], self.hidden_units[i + 1]) for i in range(len(self.hidden_units) - 1)
        ])
        for name, tensor in self.linear.named_parameters():
            if 'weight' in name:
                nn.init.normal_(tensor, mean=0, std=0.0001)

        # self.bn = nn.ModuleList([
        #     nn.Linear(self.hidden_units[i], self.hidden_units[i + 1]) for i in range(len(self.hidden_units) - 1)
        # ])
        self.activation = nn.ReLU()

    def forward(self, X):
        inputs = X
        for i in range(len(self.linear)):
            fc = self.linear[i](inputs)
            fc = self.activation(fc)
            fc = self.dropout(fc)
            inputs = fc
        return inputs


class InteractingLayer(nn.Module):
    def __init__(self, embedding_size, head_num=2, use_res=True, scaling=False):
        super(InteractingLayer, self).__init__()
        self.att_embedding_size = embedding_size // head_num
        self.head_num = head_num
        self.use_res = use_res
        self.scaling = scaling

        self.W_Query = nn.Parameter(torch.Tensor(embedding_size, embedding_size))
        self.W_Key = nn.Parameter(torch.Tensor(embedding_size, embedding_size))
        self.W_Value = nn.Parameter(torch.Tensor(embedding_size, embedding_size))

        if self.use_res:
            self.W_Res = nn.Parameter(torch.Tensor(embedding_size, embedding_size))
        for tensor in self.parameters():
            nn.init.normal_(tensor, mean=0.0, std=0.05)

    def forward(self, inputs):

        # inputs: [1024, 26, 4]
        #keys: [1024, 26, 4]
        querys = torch.tensordot(inputs, self.W_Query, dims=([-1], [0]))
        keys = torch.tensordot(inputs, self.W_Key, dims=([-1], [0]))
        values = torch.tensordot(inputs, self.W_Value, dims=([-1], [0]))

        # keys: [2, 1024, 26, 2]
        querys = torch.stack(torch.split(querys, self.att_embedding_size, dim=2))
        keys = torch.stack(torch.split(keys, self.att_embedding_size, dim=2))
        values = torch.stack(torch.split(values, self.att_embedding_size, dim=2))

        # inner_product: [2, 1024, 26, 26]
        inner_product = torch.einsum('bnik,bnjk->bnij', querys, keys)

        if self.scaling:
            inner_product /= self.att_embedding_size ** 0.5
        self.normalized_att_scores = F.softmax(inner_product, dim=-1)

        # [2, 1024, 26, 2]
        result = torch.matmul(self.normalized_att_scores, values)
        # [1, 1024, 26, 4]
        result = torch.cat(torch.split(result, 1, ), dim=-1)
        # [1024, 26, 4]
        result = torch.squeeze(result, dim=0)
        if self.use_res:
            result += torch.tensordot(inputs, self.W_Res, dims=([-1], [0]))
        result = F.relu(result)
        return result

class AutoInt(nn.Module):
    def __init__(self, feat_size, embedding_size, linear_feature_columns, dnn_feature_columns, att_layer_num=3,
                 att_head_num=2,
                 att_res=True, dnn_hidden_units=(256, 128)):
        super(AutoInt, self).__init__()
        self.sparse_feature_columns = list(filter(lambda x: x[1] == 'sparse', dnn_feature_columns))
        self.embedding_dic = nn.ModuleDict({
            feat[0]: nn.Embedding(feat_size[feat[0]], embedding_size, sparse=False) for feat in
            self.sparse_feature_columns
        })
        self.dense_feature_columns = list(filter(lambda x: x[1] == 'dense', dnn_feature_columns))

        self.feature_index = defaultdict(int)
        start = 0
        for feat in feat_size:
            self.feature_index[feat] = start
            start += 1

        self.dnn = DNN(len(self.dense_feature_columns) + embedding_size * len(self.embedding_dic), dnn_hidden_units,
                       0.5)

        self.dnn_linear = nn.Linear(dnn_hidden_units[-1] + embedding_size * len(self.embedding_dic), 1, bias=False)

        for name, tensor in self.dnn_linear.named_parameters():
            if 'weight' in name:
                nn.init.normal_(tensor, mean=0, std=0.00001)

        dnn_hidden_units = [len(feat_size), 1]
        self.linear = nn.ModuleList([
            nn.Linear(dnn_hidden_units[i], dnn_hidden_units[i + 1]) for i in range(len(dnn_hidden_units) - 1)
        ])
        for name, tensor in self.linear.named_parameters():
            if 'weight' in name:
                nn.init.normal_(tensor, mean=0, std=0.00001)

        self.int_layers = nn.ModuleList([
            InteractingLayer(embedding_size, att_head_num, att_res) for _ in range(att_layer_num)
        ])

        self.out = nn.Sigmoid()
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, X):
        # # X [1024, 39]
        # logit = X
        # for i in range(len(self.linear)):
        #     fc = self.linear[i](logit)
        #     fc = self.act(fc)
        #     fc = self.dropout(fc)
        #     logit = fc
        # logit [1024, 1]
        sparse_embedding = [
            self.embedding_dic[feat[0]](X[:, self.feature_index[feat[0]]].long()).reshape(X.shape[0], 1, -1)
            for feat in self.sparse_feature_columns]
        sparse_input = torch.cat(sparse_embedding, dim=1)
        sparse_input = torch.flatten(sparse_input, start_dim=1)
        dense_values = [X[:, self.feature_index[feat[0]]].reshape(-1, 1) for feat in self.dense_feature_columns]
        dense_input = torch.cat(dense_values, dim=1)

        # att_input [1024, 26, 4]
        att_input = torch.cat(sparse_embedding, dim=1)
        for layer in self.int_layers:
            att_input = layer(att_input)
        # att_out [1024, 104]
        att_output = torch.flatten(att_input, start_dim=1)
        #print('att_output shape', att_output.shape)

        # dnn_input [1024, 117] 26*4+13
        dnn_input = torch.cat((dense_input, sparse_input), dim=1)
        # deep_out [1024, 128]
        deep_out = self.dnn(dnn_input)
        stack_out = torch.cat((att_output, deep_out), dim=-1)
        #print('stack_out shape', stack_out.shape)
        # logit += self.dnn_linear(stack_out)
        final= self.dnn_linear(stack_out)
        #print('logit shape', logit.shape)
        y_pred = torch.sigmoid(final)
        return y_pred


if __name__ == '__main__':
    batch_size =1024
    lr = 1e-3
    wd = 1e-5
    epoches = 40
    seed = 2022
    embedding_size =10
    device = 'cuda:0'
    # pd.set_option('display.max_rows', None)  # 显示数据中所有的列
    data = pd.read_csv('vehicle_data_model_50w_3.csv')
    # data = pd.read_csv('vehicle_data_model_30w_2.csv')
    dense_feature= ['driver_auth_success_days','cargo_search_cnt_3','cargo_search_cnt_7','scan_cargo_cnt_3','scan_cargo_cnt_7','click_cargo_cnt_3_x','click_cargo_cnt_7','call_cnt_3_driver','call_cnt_7_driver',
               'shipper_auth_success_days','exposure_cargo_cnt_3','exposure_cnt_3','click_cargo_cnt_3_y','click_cnt_3','cargo_weight','vector_regular_subscribe_line',
                       'vector_regular_cargo_line_all','vector_regular_cargo_truck_type_all','vector_regular_cargo_truck_length_all','vector_regular_cargo_line_30',
                       'vector_regular_cargo_truck_type_30','vector_regular_cargo_truck_length_30']


    # 假设你的数据集中包含'label'列，并且dense_feature已经定义
    sparse_feature  = data.drop(columns=['label'] + dense_feature).columns.tolist()
    print(len(sparse_feature))
    pd.options.display.max_rows = None  # 显示所有列
    data[sparse_feature]=data[sparse_feature].astype('uint8')
    target = ['label']

    feat_sizes = {}  # 初始化一个空字典 feat_sizes。
    feat_sizes_dense = {feat: 1 for feat in dense_feature}#这里将稠密特征的维度大小设置为1，因为这些特征不需要经过 Embedding 层，直接作为输入。
    # 对每个稀疏特征创建一个键值对，键为特征名称，值为该特征在数据中唯一取值的数量（即不同的类别个数）。
    feat_sizes_sparse = {feat: len(data[feat].unique()) for feat in sparse_feature}
    # 将稠密特征和稀疏特征的维度大小更新到 feat_sizes 字典中，得到包含所有特征维度大小信息的字典 feat_sizes。
    feat_sizes.update(feat_sizes_dense)
    feat_sizes.update(feat_sizes_sparse)
    # for feat in sparse_feature:
    #     lbe = LabelEncoder()  #使用LabelEncoder类对每个稀疏特征进行编码,将每个特征的字符串类型的值映射成整数编号
    #     data[feat] = lbe.fit_transform(data[feat])


    # 定义fixlen_feature_columns，包含了所有特征的名称和类型（sparse或dense）。
    fixlen_feature_columns = [(feat, 'sparse') for feat in sparse_feature] + [(feat, 'dense') for feat in dense_feature]
    dnn_feature_columns = fixlen_feature_columns
    linear_feature_columns = fixlen_feature_columns

   
    # 数据集划分
    train, test = train_test_split(data, test_size=0.2, random_state=seed)
    validation, test = train_test_split(test, test_size=0.5, random_state=seed)

    # DataLoader准备
    def create_data_loader(df, batch_size):
        labels = pd.DataFrame(df['label'])
        features = df.drop(columns=['label'])
        tensor_data = TensorDataset(torch.from_numpy(np.array(features)), torch.from_numpy(np.array(labels)))
        return DataLoader(tensor_data, shuffle=True, batch_size=batch_size)


    train_loader = create_data_loader(train, batch_size)
    validation_loader = create_data_loader(validation, batch_size)
    test_loader = create_data_loader(test, batch_size)

    # 模型初始化
    # model = FiBiNET(feat_sizes, embedding_size, linear_feature_columns, dnn_feature_columns)
    device = 'cuda:0'
    model = AutoInt(feat_sizes, embedding_size, linear_feature_columns, dnn_feature_columns).to(device)
    loss_func = nn.BCELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    # 早停策略参数
    early_stopping_threshold = 0.80
    best_validation_auc = 0
    
    # 定义一个函数来计算给定数据加载器上的平均损失和AUC
    def evaluate(loader):
        model.eval()
        total_loss = 0.0
        total_samples = 0
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device).float(), y.to(device).float()
                y_hat = model(x)
                loss = loss_func(y_hat, y)
                total_loss += loss.item() * x.size(0)
                total_samples += x.size(0)
        avg_loss = total_loss / total_samples
        auc = get_auc(loader, model)
        return avg_loss, auc


    # 训练循环
    for epoch in range(epoches):
        total_loss_epoch = 0.0
        total_tmp = 0
        model.train()
        for index, (x, y) in enumerate(train_loader):
            x, y = x.to(device).float(), y.to(device).float()
            y_hat = model(x)

            optimizer.zero_grad()
            loss = loss_func(y_hat, y)
            loss.backward()
            optimizer.step()
            total_loss_epoch += loss.item()
            total_tmp += 1

        # 验证集评估
        validation_loss, validation_auc = evaluate(validation_loader)
        print(
            f'Epoch {epoch}/{epoches}, Train Loss: {total_loss_epoch / total_tmp:.4f}, Validation_loss: {validation_loss:.4f},Validation AUC: {validation_auc:.4f}')

        # 更新最佳验证集AUC
        if validation_auc > best_validation_auc:
            best_validation_auc = validation_auc

        # 早停判断
        if validation_auc >= early_stopping_threshold:
            print(f'Early stopping triggered at epoch {epoch}, Validation AUC: {validation_auc:.4f}')
            break

    # 测试集评估
    test_loss, final_test_auc = evaluate(test_loader)
    print(f'Test_loss:{test_loss:.4f},Final Test AUC: {final_test_auc:.4f}')

17
Epoch 0/40, Train Loss: 0.2527, Validation_loss: 0.2301,Validation AUC: 0.6836
Epoch 1/40, Train Loss: 0.2260, Validation_loss: 0.2235,Validation AUC: 0.7230
Epoch 2/40, Train Loss: 0.2222, Validation_loss: 0.2208,Validation AUC: 0.7355
Epoch 3/40, Train Loss: 0.2198, Validation_loss: 0.2181,Validation AUC: 0.7481
Epoch 4/40, Train Loss: 0.2175, Validation_loss: 0.2176,Validation AUC: 0.7511
Epoch 5/40, Train Loss: 0.2158, Validation_loss: 0.2166,Validation AUC: 0.7563
Epoch 6/40, Train Loss: 0.2145, Validation_loss: 0.2137,Validation AUC: 0.7618
Epoch 7/40, Train Loss: 0.2129, Validation_loss: 0.2145,Validation AUC: 0.7637
Epoch 8/40, Train Loss: 0.2116, Validation_loss: 0.2118,Validation AUC: 0.7672
Epoch 9/40, Train Loss: 0.2103, Validation_loss: 0.2113,Validation AUC: 0.7698
Epoch 10/40, Train Loss: 0.2093, Validation_loss: 0.2098,Validation AUC: 0.7733
Epoch 11/40, Train Loss: 0.2084, Validation_loss: 0.2094,Validation AUC: 0.7764
Epoch 12/40, Train Loss: 0.2071, Validation_los